In [ ]:
import random                                                                                   # 1) Импорт библиотек.
import sys
import time
import tkinter as tk
import pygame as pg
import random as rnd
from itertools import permutations

window = tk.Tk()                                                                          # 2) Создание окна для ввода.
window.title('Курсовой проект "Шахматы"')
window.geometry('335x150')
window.resizable(False, False)

n, n_entry = tk.Label(master=window, text='Размер доски (N):'), tk.Entry(width=10)         # 3) Заполнение. Окно ввода.
n.pack()
n_entry.pack()
n_entry.focus()
l, l_entry = tk.Label(master=window, text='Фигур надо поставить (L):'), tk.Entry(width=10)
l.pack()
l_entry.pack()
k, k_entry = tk.Label(master=window, text='Фигур уже расставлено (K):'), tk.Entry(width=10)
k.pack()
k_entry.pack()
correct, N, L, K = False, 0, 0, 0

def error():                                                                        # 4) Метод, вызывающий окно ошибки.
    error_window = tk.Tk()
    error_window.title('Ошибка!')
    error_window.geometry('230x47')
    tk.Label(master=error_window, text='Неверный формат ввода!').pack()
    tk.Button(master=error_window, text='Ок', command= lambda: error_window.destroy()).pack()
    error_window.mainloop()

def enter():                                # 5) Обработка ввода N, L, K, формирует окно с ошибкой, если формат не тот.
    global correct, N, L, K
    N, L, K = n_entry.get().strip(), l_entry.get().strip(), k_entry.get().strip()
    window.destroy()
    if not N.isdigit() or not L.isdigit() or not K.isdigit():
        error()
    else:
        N, L, K = int(N), int(L), int(K)
        if 1 <= N <= 20 and L >= 0 and K >= 0:
            correct = True

enter_button = tk.Button(text="Ввод", command=enter)
enter_button.pack()
window.mainloop()

if not correct:                                                     # 6) Проверка, что функция enter не увидела ошибок.
    raise TypeError('неверный тип вводных данных')


def enter_coords(entry_fields: list):                                               # 7) Обработка введённых координат.
    for e in enter_array:
        string = e.get()
        coords_pair = tuple(string.split())
        if len(coords_pair) == 2 and False not in tuple(map(lambda x: x.isdigit(), coords_pair)):
            coords.append((int(coords_pair[0]), int(coords_pair[1])))
        else:
            error()
    k_window.destroy()


coords = []                                                                    # 8) Подготовка окна с вводом координат.
k_window = tk.Tk()
k_window.geometry(f'250x{20 * (K + 1)}')
k_window.title('Координаты')
enter_array = [tk.Entry(master=k_window) for o in range(K)]
for i in range(K):
    enter_array[i].pack()

enter_button = tk.Button(text="Ввод", command=lambda: enter_coords(enter_array))         # 9) Кнопка ввода, запускающая
enter_button.pack()                                                                      # функцию 7).
if K > 0:
    k_window.mainloop()


def beaten_cells(size: int, x: int, y: int):      # 13) Функция, ищет клетки под боем для Конь-Дракона (Король + Слон):
    beaten = []
    for row in range(-1, 2):                                                                   # [Клетки вокруг фигуры]
        for col in range(-1, 2):
            if 0 <= x + row < size and 0 <= y + col < size and (x + row, y + col) not in beaten:
                beaten.append((x + row, y + col))
    step = 2                                                                                    # [Клетки по диагонали]
    while step < size:
        for row in range(x - step, x + step + 1, step * 2):
            for col in range(y - step, y + step + 1, step * 2):
                if 0 <= row < size and 0 <= col < size and (row, col) not in beaten:
                    beaten.append((row, col))
        step += 1
    return beaten

all_coords = list((m, n) for m in range(N) for n in range(N))                  # 10) Подготовка к вычислительной части.
beaten_coords = []
                                                                            # 11) Получение координат под боем K фигур.
for fig in coords:
    if fig not in beaten_coords:
        beaten_coords.extend(beaten_cells(N, fig[0], fig[1]))
        beaten_coords = list(set(beaten_coords))
    else:
        error()

beaten_coords.sort()
not_beaten_coords = list(filter(lambda x: x not in beaten_coords, all_coords))          # 12) -||- не под боем K фигур.

solution_list = []                                                     # 13) Перебор вариантов методом бинарной строки.
for variant in set(map(lambda x: ''.join(list(x)), set(permutations(('1' * L).zfill(len(not_beaten_coords)))))):
    valid = True
    start_index = 0
    temp_beaten_coords = beaten_coords[:]
    new_coords = []
    for i in range(L):
        index = variant.find('1', start_index, len(variant))
        if not_beaten_coords[index] not in temp_beaten_coords:
            temp_beaten_coords += beaten_cells(N, not_beaten_coords[index][0], not_beaten_coords[index][1])
            new_coords.append(not_beaten_coords[index])
        else:
            valid = False
            break
        start_index = index + 1
    if valid:
        solution_list.append(coords + new_coords)


def output():                                                                 # 14) Создание окна с кнопкой для вывода.
    out_file = open('output.txt', 'w')
    for valid_variant in solution_list:
        out_file.write(str(valid_variant)[1:-1] + '\n')
    output_window.destroy()


output_window = tk.Tk()
output_window.geometry('275x20')
output_window.title("Вывод данных")
output_button = tk.Button(master=output_window, text="Вывести", command=output)
output_button.pack()
output_window.mainloop()

colors = {'white': (255, 255, 255),                                         # 15) Подготовка к запуску шахматной доски.
          'black': (0, 0, 0),
          'gray': (128, 128, 128),
          'red': (200, 25, 25),
          'blue': (25, 25, 200),
          'yellow': (255, 255, 0)}
chess = pg.display.set_mode((800, 800))
pg.display.set_caption('Шахматы')
chess.fill(colors['gray'])
border_width = 2
cell_size = (800 - (N + 1) * border_width) / N
clock = pg.time.Clock()


class Square(pg.sprite.Sprite):                                                                 # 16) Класс для клетки.
    def __init__(self, c_size: float, color: str):
        super(Square, self).__init__()
        self.surf = pg.Surface((c_size, c_size))
        if color == 'white':
            self.surf.fill(colors['white'])
        else:
            self.surf.fill(colors['black'])
        self.rect = self.surf.get_rect()


class Figure(pg.sprite.Sprite):                                                                 # 17) Класс для фигуры.
    def __init__(self, x: int, y: int, type: str):
        self.x = x
        self.y = y
        self.type = type
        self.surf = pg.Surface((cell_size, cell_size))
        if type == 'figure':
            self.surf.fill(colors['red'])
            self.color = colors['red']
        elif type == 'new figure':
            self.surf.fill(colors['yellow'])
            self.color = colors['yellow']
        else:
            self.surf.fill(colors['blue'])
            self.color = colors['blue']
        self.rect = self.surf.get_rect()
        self.pygame_x = border_width * (x + 1) + cell_size * self.x
        self.pygame_y = border_width * (y + 1) + cell_size * self.y

    def __repr__(self):
        if self.type == 'figure' or self.type == 'new figure':
            return 'Figure'
        else:
            return 'Beaten Cell'

game_on = True
random_variant = random.choice(solution_list)

while game_on:                                                                                # 18) Визуализация доски.
    clock.tick(50)
    c_array = []
    c_coords_array = []
    for x in range(N):                                                                 # [Отрисовка чёрно-белых клеток]
        c_array.append([])
        c_coords_array.append([])
        for y in range(N):
            coord_x, coord_y = border_width * (x + 1) + cell_size * x, border_width * (y + 1) + cell_size * y
            if (x + y) % 2 == 0:
                c_array[x].append(Square(cell_size, 'white').surf)
            else:
                c_array[x].append(Square(cell_size, 'black').surf)
            c_coords_array[x].append((coord_x, coord_y, coord_x + cell_size, coord_y + cell_size))
            chess.blit(source=c_array[x][y], dest=(coord_x, coord_y))
    count = 0
    for x_and_y in random_variant:                                          # [Отрисовка случайного найденного решения]
        for new_beaten_cell in beaten_cells(N, x_and_y[0], x_and_y[1]):
            cell = Figure(new_beaten_cell[0], new_beaten_cell[1], 'beaten cell')
            chess.blit(source=cell.surf, dest=(cell.pygame_x, cell.pygame_y))
        if count >= K:
            fig = Figure(x_and_y[0], x_and_y[1], 'new figure')
        else:
            fig = Figure(x_and_y[0], x_and_y[1], 'figure')
        chess.blit(source=fig.surf, dest=(fig.pygame_x, fig.pygame_y))
        count += 1
    pg.display.flip()                                                                            # [Обновление дисплея]

    for i in pg.event.get():                                                                      # 19) Выход из цикла.
        if i.type == pg.QUIT:
            game_on = False